# IMC Lung Disease Data Domain Classification using UTAG

## Loading Libraries and Data

In [1]:
cd ~/imc-graph/gatdu

C:\Users\jkim0\imc-graph\gatdu


In [2]:
from gatdu.segmentation import segment_slide, evaluate_performance
from gatdu.vizualize import add_spatial_image, adj2chord
from gatdu.utils import domain_connectivity, celltype_connectivity, evaluate_clustering

import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt

sc.settings.set_figure_params(dpi=150, fontsize= 6, dpi_save = 300)

Warning, OS could not be easily identified. Using default dir ~/.gatdu to store resources but that might not work!


In [3]:
cd ~/imc-graph/covid-imc1

C:\Users\jkim0\imc-graph\covid-imc1


In [6]:
#
adata = sc.read('gatdu_adata.h5ad')
adata

View of AnnData object with n_obs × n_vars = 627125 × 37
    obs: 'sample', 'roi', 'obj_id', 'X_centroid', 'Y_centroid', 'area', 'perimeter', 'major_axis_length', 'eccentricity', 'solidity'
    obsm: 'spatial'

In [5]:
results = sc.read('gatdu_results.h5ad')

## UTAG Preprocessing

In [8]:
adata = sc.read('processed/quantification.filtered.h5ad')
adata = adata[:,~adata.var.index.str.contains('190BCKG')]

sc.pp.log1p(adata)
z_score_cap = 3.0
batch_key = 'sample'
sc.pp.scale(adata, max_value=z_score_cap)
adata.X[adata.X < -z_score_cap] = -z_score_cap

#sc.external.pp.mnn_correct(adata, key = batch_key)
sc.pp.combat(adata, key = batch_key)
sc.pp.scale(adata, max_value=z_score_cap)
adata.X[adata.X < -z_score_cap] = -z_score_cap

C:\Users\jkim0\anaconda3\envs\imc-graph\lib\site-packages\scanpy\preprocessing\_simple.py:373: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)


# Running UTAG Algorithm

In [ ]:
%%time
results = segment_slide(adata, slide_key = 'roi', max_dist = 20, normalize_adjacency_matrix = True, batch_key = 'sample', apply_clustering = True, clustering_method = 'parc')

In [19]:
adata.write('gatdu_adata_parc.h5ad')

In [18]:
results.write('utag_results_parc.h5ad')

## Visualizing Results

In [4]:
adata = sc.read('gatdu_adata_parc.h5ad')
results = sc.read('utag_results_parc.h5ad')

In [ ]:
import seaborn as sns

cm = sns.clustermap(results.to_df().corr(), cmap = 'coolwarm', vmin = -1, vmax = 1)
var_order = results.var.iloc[cm.dendrogram_col.reordered_ind].index

In [ ]:
#sc.pl.StackedViolin(result, var_names = result.var.index, groupby='cluster', dendrogram = True, cmap = 'coolwarm').legend(title = 'Median').show()
#plt.figure(dpi = 300, figsize = (2000,2000))

results.obs['UTAG Label'] = pd.Categorical(results.obs['UTAG Label_parc_0.3'])
adata.obs['UTAG Label'] = results.obs['UTAG Label']
adata.uns['UTAG Label_colors'] = (sns.color_palette('tab10').as_hex() + sns.color_palette('tab20b').as_hex() + sns.color_palette('tab20c').as_hex())[:len(adata.obs['UTAG Label'].unique())]
results.uns['UTAG Label_colors'] = adata.uns['UTAG Label_colors']

sc.settings.set_figure_params(dpi=300, fontsize= 10)
#sc.pl.matrixplot(results, var_names = var_order, groupby='UTAG Label', cmap = 'coolwarm', vmin = -1, vmax = 1, show = True, save = 'unlabelled_matrixplot')
mp = sc.pl.matrixplot(adata, var_names = var_order, groupby='UTAG Label',vmin = -1, vmax = 1, return_fig=True)
mp.add_totals().style(edge_color='black', cmap = 'coolwarm').savefig('figures/unlabelled_matrixplot_0.1.pdf')

## Labelling Results

In [5]:
import numpy as np
import seaborn as sns

UTAG_parc_1_mapper_broad = {
    0: 'Airway Wall',
    1: 'Airway Wall',
    2: 'Alveolar',
    3: 'Airway Wall',
    4: 'Epithelial Cell',
    5: 'Vessel',
    6: 'Immune Cell',
    7: 'Airway Wall',
    8: 'Immune Cell',
    9: 'Immune Cell',
    10: 'Vessel',
    11: 'Immune Cell',
    12: 'Airway Wall',
    13: 'Epithelial Cell',
    14: 'Immune Cell',
    15: 'Immune Cell',
    16: 'Immune Cell',
    17: 'Submucousal Gland',
    18: 'Cartillage',
    19: None
}


adata.obs['UTAG Label'] = pd.Categorical(results.obs['UTAG Label_parc_0.3'].replace(UTAG_parc_1_mapper_broad))
results.obs['UTAG Label'] = adata.obs['UTAG Label'] 

adata = adata[~adata.obs['UTAG Label'].isna()]
results = results[~results.obs['UTAG Label'].isna()]

adata.uns['UTAG Label_colors'] = np.array(sns.color_palette('tab10').as_hex() + sns.color_palette('tab20b').as_hex() + sns.color_palette('tab20c').as_hex()).astype('object')
results.uns['UTAG Label_colors'] = np.array(sns.color_palette('tab10').as_hex() + sns.color_palette('tab20b').as_hex() + sns.color_palette('tab20c').as_hex()).astype('object')

adata.uns['UTAG Label_colors'] = adata.uns['UTAG Label_colors'][:adata.obs['UTAG Label'].unique().shape[0]]
results.uns['UTAG Label_colors'] = results.uns['UTAG Label_colors'][:results.obs['UTAG Label'].unique().shape[0]]
adata.write('utag_adata_parc_0.3_annotated2.h5ad')
results.write('utag_results_parc_0.3_annotated2.h5ad')

Trying to set attribute `.uns` of view, copying.
Trying to set attribute `.uns` of view, copying.


In [51]:
adata.uns['UTAG Label_colors'] = sns.color_palette('tab10').as_hex()[:len(adata.obs['UTAG Label'].unique())]
results.uns['UTAG Label_colors'] = adata.uns['UTAG Label_colors']

In [9]:
results.obs['UTAG Label'] = pd.Categorical(results.obs['UTAG Label'], categories = ['Epithelial Cell', 'Airway Wall', 'Alveolar', 'Immune Cell', 'Submucousal Gland', 'Vessel', 'Cartillage'])

In [6]:
mp = sc.pl.matrixplot(results, var_names = var_order, groupby='UTAG Label', vmin = -1, vmax = 1, return_fig=True)
mp.add_totals().style(edge_color='black', cmap = 'coolwarm').savefig('figures/labelled_matrixplot2.pdf')

NameError: name 'var_order' is not defined

In [79]:
adata.write('UTAG_adata.h5ad')
results.write('UTAG_results.h5ad')

# Visualizing of UTAG Clusters

In [4]:
adata = sc.read('UTAG_adata.h5ad')
results = sc.read('UTAG_results.h5ad')

In [28]:
adata

AnnData object with n_obs × n_vars = 627125 × 37
    obs: 'sample', 'roi', 'obj_id', 'X_centroid', 'Y_centroid', 'area', 'perimeter', 'major_axis_length', 'eccentricity', 'solidity', 'UTAG Label'
    var: 'mean', 'std'
    uns: 'log1p', 'UTAG Label_colors'
    obsm: 'spatial'

In [10]:
def add_scale_box_to_fig(
    img,
    ax,
    box_width: int = 200,
    box_height: float = 2,
    color: str = 'white'
):    
    import matplotlib.patches as patches
    x = img.shape[1]
    y = img.shape[0]
    
    # Create a Rectangle patch
    rect = patches.Rectangle((x - box_width, y * (1-box_height/100)), box_width, y * (box_height/100), linewidth=0.2, edgecolor='black', facecolor=color)

    # Add the patch to the Axes
    ax.add_patch(rect)
    return ax

In [12]:
from tqdm import tqdm
sc.settings.set_figure_params(dpi=150, fontsize= 6, dpi_save = 300)

point_size = 6
i = 0
for slide in tqdm(results.obs['roi'].unique()):
    result_batch = results[results.obs['roi'] == slide].copy()
    ad_batch = adata[adata.obs['roi'] == slide].copy()
    sample = result_batch.obs['sample'].unique()[0]
    fig, axs = plt.subplots(1, 2, figsize = (10,4), dpi = 300)
    result_batch = add_spatial_image(adata = result_batch,
                          image_path = f'processed//{sample}/tiffs/{slide}_full.tiff',
                          rgb_channels = [36, 4, 41], # Keratin 8/18, aSMA, DNA2
                          gamma = 1,
                          gain = 0.2,
                          log_transform = True)
    
    sc.pl.spatial(result_batch, spot_size = 10, ax = axs[0], show = False, title = f'ROI: {slide}', frameon = False)
    #sc.pl.spatial(ad_batch, color = 'leiden', spot_size = 10, ax = axs[1], show = False, title = 'Celltype Labels on Image', frameon = False, legend_loc = 'on data', legend_fontoutline = 0, legend_fontsize = 0)
    sc.pl.spatial(result_batch, color = 'UTAG Label', spot_size = 15, ax = axs[1], show = False, title = 'UTAG Labels on Image', frameon = False, legend_fontsize = 5, alpha_img = 0)
    
    add_scale_box_to_fig(result_batch.uns["spatial"]['image']["images"]["hires"], axs[0])
    plt.tight_layout()
    #plt.show()
    plt.savefig(f'figures/{slide}_utag_0.3_labelled2.png')
    plt.close()
    i += 1

100%|██████████| 237/237 [10:04<00:00,  2.55s/it]


In [27]:
results.obs['disease'] = results.obs['sample'].str.replace('[0-9]*','').str.replace('[0-9]*','').str[1:]

C:\Users\jkim0\AppData\Local\Temp/ipykernel_10188/293749224.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  results.obs['disease'] = results.obs['sample'].str.replace('[0-9]*','').str.replace('[0-9]*','').str[1:]


In [ ]:
point_size = 4
i = 0
for slide in result.obs['roi'].unique():
    if i > 30:
        break
    if 'LATE' in slide:
        result_batch = result[result.obs['roi'] == slide].copy()
        ad_batch = adata[adata.obs['roi'] == slide].copy()
        sample = result_batch.obs['sample'].unique()[0]
        fig, axs = plt.subplots(1, 5, figsize = (20,4), dpi = 300)
        result_batch = add_spatial_image(adata = result_batch,
                              image_path = f'processed//{sample}/tiffs/{slide}_full.tiff',
                              YX_coordinates = result_batch.obs[['Y_centroid', 'X_centroid']],
                              rgb_channels = [36, 4, 41], # SARSSpikeS1, aSMA, DNA
                              contrast_percentile = (0, 98),
                              log_transform = True)

        sc.pl.spatial(result_batch, spot_size = 10, ax = axs[0], show = False, title = f'ROI: {slide}', frameon = False)
        sc.pl.umap(ad_batch, color = 'leiden', ax = axs[1], show = False, title = 'Celltype UMAP', frameon = False, size = point_size, legend_loc = 'on data', legend_fontoutline = 1, legend_fontsize = 3)
        sc.pl.spatial(ad_batch, color = 'leiden', spot_size = 10, ax = axs[2], show = False, title = 'Celltype Labels on Image', frameon = False, legend_loc = 'on data', legend_fontoutline = 0, legend_fontsize = 0)
        sc.pl.umap(result_batch, color = 'UTAG Label', ax = axs[3], show = False, title = 'UTAG UMAP', frameon = False, size = point_size, legend_loc = 'on data', legend_fontoutline = 1, legend_fontsize = 3)
        sc.pl.spatial(result_batch, color = 'UTAG Label', spot_size = 10, ax = axs[4], show = False, title = 'UTAG Labels on Image', frameon = False, legend_fontsize = 5, alpha_img = 0)

        #plt.tight_layout()
        plt.show()
        i += 1